In [1]:
# Only with gooogle colab 
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/Travail/Hackathon/code/src
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers 
import pandas as pd
import pickle
from WeatherDataset import WeatherDataset
import tensorflow as tf

/content/drive/MyDrive/Travail/Hackathon/code/src


In [3]:
path = "/content/drive/MyDrive/Travail/Hackathon/dataset/dataset_reoganized_extended_v1_noe.pkl"
f = open(path, 'rb')
df = pickle.load(f)
f.close()

In [5]:
x, y1, y2 = df
x.shape, y1.shape, y2.shape

((21966, 17), (21965, 5, 6), (21965, 5, 1))

In [7]:
train_dataset = WeatherDataset(sequence_length=200, df=df, batch_size=2, instanciate_data=False)
train_dataset.import_data(x, y1, y2)
train_dataset.create_dataset()

In [ ]:
import numpy as np


def readucr(filename):
    data = np.loadtxt(filename, delimiter="\t")
    y = data[:, 0]
    x = data[:, 1:]
    return x, y.astype(int)


root_url = "https://raw.githubusercontent.com/hfawaz/cd-diagram/master/FordA/"

x_train, y_train = readucr(root_url + "FordA_TRAIN.tsv")
x_test, y_test = readucr(root_url + "FordA_TEST.tsv")

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

n_classes = len(np.unique(y_train))

idx = np.random.permutation(len(x_train))
x_train = x_train[idx]
y_train = y_train[idx]

y_train[y_train == -1] = 0
y_test[y_test == -1] = 0


In [22]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res


In [34]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    weather_type = [layers.Dense(6, activation="softmax", name=f"weather_type_softmax_{i}")(x) for i in range(5)]
    rain_probabilities = layers.Dense(5, activation='sigmoid', name='rain_prob')(x)
    return  keras.Model(inputs, [*weather_type, rain_probabilities])
    # outputs = layers.Dense(n_classes, activation="softmax")(x)
    # return keras.Model(inputs, outputs)


In [41]:
input_shape = (train_dataset.sequence_length, train_dataset.n_features)

model = build_model(
    input_shape,
    head_size=1024,
    # head_size=256,
    num_heads=4,
    ff_dim=4,
    # num_transformer_blocks=4,
    num_transformer_blocks=16,
    # mlp_units=[128],
    mlp_units=[128, 128, 128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3),
                       loss = {"weather_type_softmax_0":"categorical_crossentropy",
                               "weather_type_softmax_1":"categorical_crossentropy",
                               "weather_type_softmax_2":"categorical_crossentropy",
                               "weather_type_softmax_3":"categorical_crossentropy",
                               "weather_type_softmax_4":"categorical_crossentropy",
                               "rain_prob":'mae'}
                       )

# model.compile(
#     loss="sparse_categorical_crossentropy",
#     optimizer=keras.optimizers.Adam(learning_rate=1e-4),
#     metrics=["sparse_categorical_accuracy"],
# )

model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    train_dataset.data,
    # validation_split=0.2,
    epochs=20,
    # batch_size=64,
    # callbacks=callbacks,
)

model.evaluate(train_dataset.data, verbose=1)


Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, 200, 17)]    0           []                               
                                                                                                  
 layer_normalization_104 (Layer  (None, 200, 17)     34          ['input_15[0][0]']               
 Normalization)                                                                                   
                                                                                                  
 multi_head_attention_52 (Multi  (None, 200, 17)     290833      ['layer_normalization_104[0][0]',
 HeadAttention)                                                   'layer_normalization_104[0][0]']
                                                                                           

[3.5347447395324707,
 0.6654771566390991,
 0.7611979246139526,
 0.7678189277648926,
 0.7179892659187317,
 0.5929391980171204,
 0.029321037232875824]